In [1]:
# basic
import argparse
import os,datetime

# save things
import pandas as pd
from keras.callbacks import CSVLogger

# model_func
from tools.model_func import *

Using TensorFlow backend.
W0824 19:57:24.484020 140437491287872 deprecation_wrapper.py:119] From /home/angela/xmtc/tools/model_func.py:21: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""

# copied code

In [3]:
import tensorflow_hub as hub
from bert.tokenization import FullTokenizer
from keras import backend as K
from keras.layers import Layer

In [4]:
from tools.MyClock import MyClock
clk = MyClock()

In [5]:
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

# convert data for bert

In [6]:
import pickle

In [10]:
# tokenizer sequence to text keep <UNK\>
IN_DIR = 'data/amazon_hierarchy_2'
PKL_DIR = 'data/amazon_hierarchy.pkl'

x_trains,_,x_tests,_ = get_input(in_dir=IN_DIR,mode='cat',get_output=[1,0,1,0])

In [15]:
x_train = x_trains[0]
x_test = x_tests[0]
del x_trains
del x_tests

In [16]:
max_sequence_length = x_train.shape[1]

In [12]:
# get tokenizer
tok = pickle.load(open(os.path.join(IN_DIR,'tokenizer.pkl'),'rb'))
tok.index_word[0]=''
tok.index_word[1] = "w"*201 # trigger [UNK] in WordpieceTokenizer

In [13]:
sess = tf.Session()
tok_bert = create_tokenizer_from_hub_module()

W0824 19:58:12.614794 140437491287872 deprecation_wrapper.py:119] From /home/angela/xmtc/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [18]:
xx = np.concatenate([x_train,x_test])
x_gen = tok.sequences_to_texts_generator(xx)
N = xx.shape[0]
rows = []
clk.tic()
for i,text in enumerate(x_gen):
    dd = {}
    text = text.strip()
    tokens = tok_bert.tokenize(text)[:max_sequence_length-2] # truncate
    tokenized = ["[CLS]"] + tokens +["[SEP]"] # add [CLS],[SEP]
    sequence = tok_bert.convert_tokens_to_ids(tokenized) # convert to sequence
    pad_len = max([max_sequence_length-len(sequence),0]) # pad
    # add zero padding
    # input_mask = # mask zero paddings p
    dd['text'] = text
    dd['tokenized'] = tokenized
    dd['sequence'] = sequence + [0]*pad_len
    dd['mask'] = [1]*len(sequence) + [0]*pad_len
    rows.append(dd)
    if i%(N//1000)==0:
        print("{} {:.2f}%".format(clk.toc(p=False),i/N*100),end='\r')

In [21]:
df = pd.read_pickle(PKL_DIR)
ind = df[df['train/test']=='train'].index.to_list() + df[df['train/test']=='test'].index.to_list()
for i in range(len(rows)):
    rows[i]['id']=ind[i]
    if i < x_train.shape[0]:
        rows[i]['train/test']='train'
    else:
        rows[i]['train/test']='test'

In [23]:
bert_df = pd.DataFrame.from_dict(rows)
bert_df.head()

,id,mask,sequence,text,tokenized,train/test
0,0000025240,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 5070, 1037, 3796, 2003, 12372, 2934, 199...",alan a block is emeritus professor of crime la...,"[[CLS], alan, a, block, is, emeritus, professo...",train
1,0001004743,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 3694, 1018, 1022, 1996, 2375, 1997, 1996...",grade 4 8 the law of the jungle intense compet...,"[[CLS], grade, 4, 8, the, law, of, the, jungle...",train
2,0001042335,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2520, 8101, 2003, 4235, 5240, 2004, 1996...",william shakespeare is widely regarded as the ...,"[[CLS], william, shakespeare, is, widely, rega...",train
3,0001046349,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2053, 3043, 2129, 10439, 8095, 2075, 243...",no matter how appalling malicious and pretenti...,"[[CLS], no, matter, how, app, ##all, ##ing, ma...",train
4,0001046438,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 14744, 2003, 1037, 2329, 26572, 18900, 2...",fry is a british polymath actor journalist pla...,"[[CLS], fry, is, a, british, poly, ##mat, ##h,...",train


In [24]:
bert_df.to_pickle(os.path.join(IN_DIR,'bert_x.pkl'))